In [1]:
#한국어 영어 번역 데이터셋 만들기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re

# 실제 프로젝트 AI hub 기타등등..

korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "드라마 보고 있어",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "오늘 몸이 안 좋아"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "I’m watching a drama",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "I don’t feel well today"
]

# 텍스트 전처리
# uicode 정규화 , 툭수문자처리
# NFD Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다.
unicodedata.normalize('NFD',"e ' ")

"e ' "

$\acute{e}$

단일문자 U+00E9

$e$ + $\acute{}$

$e$ + 악센트가 결합이 된 형태

Unicode 정규화란 ?  같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 가지는 경우가 있는데 이걸 동일한 내부 표현으로 바꿔주는 과정

In [2]:
def preprocess_sentence(sentence , is_korean = False):
    '''
    Unicode 정규화 , 특수문자 처리
    Args : 
        sentence : 원본
        is_korean : 한국어 여부
    return:
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD' , sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip() # .strip() 좌우공백제거
    # 안녕하세요! -> 안녕하세요 !  정규식으로 특수문자 전 후에 공백 추가
    # r"[?.!,]"   문자 중에 ?.!, 특수문자가 나오면 문자를 그룹으로 캡쳐
    # r"\1"  캡쳐한 문자 (\1) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료 토큰 추가
    sentence = '<start>' + sentence + '<end>'
    return sentence

korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

# 문장을 정수 시퀀스로 변환
# 3 토크나이저

def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축 , word to index mapping
    Args:
        sentence : 문장 리스트
    returns:
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = Tokenizer(
        filters='' , # 필터 비활성화
        oov_token = '<unk>'
    )
    # 모든 문장으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

# 한국어 영어 각가의 토크나이져 생성
korean_tokenizer = create_tokenizer(korean_processed)
english_tokenizer = create_tokenizer(english_processed)

# 단어사전의 크기를 확인
korean_vocab_size = len(korean_tokenizer.word_index)+1 # oov_token = '<unk>' 때문에 플러스 1을 해준거임
english_vocab_size = len(english_tokenizer.word_index)+1
print(f'한국어 사전 크기 : {korean_vocab_size}')
print(f'영어 사전 크기 : {english_vocab_size}')

# 정수 시퀀스 변경
def encode_sentences(tokenizer , sentneces , maxlen):
    '''
    문장을 고정 길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일 길이로 맞추는
    Args : 
        Tokenizer
        sentence : 문장리스트
        maxlen : 가장 긴 문장
    returns:
        패딩된 정수 시퀀스 배열 tf의 pad_sequence
    '''
    sequence = tokenizer.texts_to_sequences(sentneces) # [i love you] -> [1,5,7]
    
    # 길이맞추기 (패딩추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence, 
        maxlen = maxlen,
        padding='post'
    )
    return padded

# 최대 시퀀스의 길이 결정 (가장 긴 문장을 기준으로!)
max_korean_len = max( len(s.split()) for s in korean_processed )
max_english_len = max( len(s.split()) for s in english_processed )
print(f'시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')

# 인코딩 수행
korean_tensor = encode_sentences(korean_tokenizer , korean_processed , max_korean_len)
english_tensor = encode_sentences(english_tokenizer , english_processed , max_english_len)
print(f'인코딩 두번째 문장')

print(f'한국어 두번째 문장 : {korean_processed[1]}')
print(f'한국어 인코딩 문장 : {korean_tensor[1]}')

print(f'영어 두번째 문장 : {english_processed[1]}')
print(f'영어 인코딩 문장 : {english_tensor[1]}')



BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

#tensorflow Dataset 객체
# 배치처리와 셔플

dataset = tf.data.Dataset.from_tensor_slices( (korean_tensor , english_tensor) )
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE , drop_remainder=True)
print('데이터셋 준비완료')
print(f'전체 샘플 수 : {len(korean_tensor)}')
print(f'배치 사이즈 : {BATCH_SIZE}')
print(f'배치수 : {len(korean_tensor) // BATCH_SIZE}')


한국어 사전 크기 : 28
영어 사전 크기 : 31
시퀀스의 최대길이
한국어 : 4
영어 : 5
인코딩 두번째 문장
한국어 두번째 문장 : <start>오늘 날씨가 좋아요<end>
한국어 인코딩 문장 : [2 4 5 0]
영어 두번째 문장 : <start>the weather is nice today<end>
영어 인코딩 문장 : [ 3  9  2 10  4]
데이터셋 준비완료
전체 샘플 수 : 10
배치 사이즈 : 2
배치수 : 5


-- step2 ---

seq2seq 구조

Encoder

    - 입력문장 -> Embedding -> LSTM + Hidden States

Decoder

    - <start>토큰 -> Embedding->LSTM + context -> 단어예측

    - 예측단어 -> 다음입력 ->반복(<end> 나올때 까지)

In [3]:
# 2-1  Encoder 클래스
import tensorflow as tf

class Encoder(tf.keras.Model):
    '''
    입력 문장을 고차원 벡터로 압축
    Embedding -> LSTN -> Hidden States 출력
    구조 : 입력 (정수시퀀스) -> Embedding -> LSTM -> 모든 타임스텝의 출력
    '''
    def __init__(self, vocab_size , embedding_dim , enc_units , batch_size):
        '''
        Args:
            vocab_size : 단어사전 크기
            embedding_dim : 임베딩 벡터 차원
            enc_units : LMS units 수 (hidden state 차원)
            batch_size : 배치 크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        
        # Embedding Layers : 정수 -> 밀집벡터(Dense Vetor) : 작은 차원 대부분의 값이 0이 아님, 연속된 실수값 공간 효율성
        # 학습 과정을 통해 단어 간 의미적 관계 학습한다 ~ 이 말은 유사한 단어는 가까운 벡터 공간에 위치한다~
        # 단점은 훈련이 필요하고 사전학습된 모델을 사용하거나 임베딩을 학습한다.
        # 밀집벡터 하면 희소벡터가 나옴.
        # 희소벡터 : 대부분의 값이 0 / 대표적으로 원-핫벡터 , 차원이 커지고 단어간의 충돌이 없다. 즉, 단어간의 유사도 표현을 못함.
        # 밀집벡터 : 학습기반이고 Word2Vec , Glove , Embedding)
        # 희소벡터 : 규칙기반이고 one-hot , Bow - 초창기 자연어 모델임.
        
        # mask_zero = True  패딩에 마스크처리를 해서 모델이 해석하지 않게 한다.
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, embedding_dim , mask_zero = True
        )
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequences = True, # [batch , seq_len , units] 형태가 나올꺼임
            return_state = True, # (output , h , c)
            recurrent_initializer = 'glorot_uniform' #가중치 초기화
        )
    def call(self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args:
            x : 입력 시퀀스 [ batch_size, seq_len]
            hidden : 초기 hidden state(첫 호출시 0벡터)
        Returns:
            output: 모든타임스템프의 출력   [batch, seq_le, enc_units]
            state_h : 마지막 hidden state   [batch, enc_units]
            state_c : 마지막 cell state [batch,enc_units]
        '''
        # 1단계 Embedding
        # [batch, seq_len]-> [batch,seq_len, embedding_dim]
        x = self.embedding(x)

        # 2단계 : LSTM 처리
        # output : 모든타임스텝의 출력(Attenton에서 사용)
        # state_h : 마지막 hidden state(디코더 초기화)
        # state_c : 마지막 cell state(LSTM 내부 메모리)
        output, state_h ,state_c =  self.lstm(x,initial_state=hidden)
        return output,state_h,state_c
    def initialize_hidden_state(self):
        '''
        인코더의 초기 hidden state 0으로 초기화
        Returns:
            두개의 0텐서 [ batch, enc_unit] (h와c)
        '''
        return [
            tf.zeros((self.batch_size, self.enc_units  )),
            tf.zeros((self.batch_size, self.enc_units  ))
        ]
# 2-2 Encoder 테스트
embedding_dim = 2
units = 512  #LSTM unit 수
# encoder 객체
encoder = Encoder(korean_vocab_size,embedding_dim,units,BATCH_SIZE)

#샘플 데이터
sample_input_batch, sample_target_batch =  next(iter(dataset))
inital_hidden = encoder.initialize_hidden_state()

# encorder 클래스에 forward-> call 호출해서 결과를 얻어야.... why call??  우리는 모델을 funtional API 방식
# 객체를 함수처럼 사용하면 된다.
sample_output, sample_h, sample_c =  encoder(sample_input_batch, inital_hidden )

print(f'encoder 출력형태')
print(f'입력형태 {sample_input_batch.shape}')  # [batch, korean_seq_len]
print(f'입력형태 {sample_target_batch.shape}') # [batch, english_seq_len] ]
print(f'hidden state :{sample_h.shape}')        # [batch, units]
print(f'cell state :{sample_c.shape}')      # [batch, units]

units

encoder 출력형태
입력형태 (2, 4)
입력형태 (2, 5)
hidden state :(2, 512)
cell state :(2, 512)


512